In [1]:
import pandas as pd
import numpy as np
#import argparse

#parser = argparse.ArgumentParser(description='Brief description of program.')
#parser.add_argument('-d', type=str, metavar='<path>', help='input director')
#parser.add_argument('-o', type=str, metavar='<path>', help='output director')
#arg = parser.parse_args()
d = '../input/dataset'
o = '../output'

def load_data():
    # Load Ratings Data
    ratings = pd.read_csv(f'{d}/ratings.csv')
    print("Ratings Data Loaded")
    print(ratings.head())

    # Load Books Data
#    books = pd.read_csv(f'{d}/books.csv')
#    print("\nBooks Data Loaded")
#    print(books.head())

    # Load Book Tags Data
#    book_tags = pd.read_csv(f'{d}/book_tags.csv')
#    print("\nBook Tags Data Loaded")
#    print(book_tags.head())

    # Load Tags Data
#    tags = pd.read_csv(f'{d}/tags.csv')
#    print("\nTags Data Loaded")
#    print(tags.head())

    # Load To-Read Data
#    to_read = pd.read_csv(f'{d}/to_read.csv')
#    print("\nTo-Read Data Loaded")
#    print(to_read.head())

#    return ratings, books, book_tags, tags, to_rea
    return ratings


In [2]:
# Call the function to load data
#ratings, books, book_tags, tags, to_read = load_data()
ratings = load_data()

# Create the rating matrix (users x books)
num_users = ratings['user_id'].max()
num_books = ratings['book_id'].max()

# Initialize a matrix of zeros
rating_matrix = np.zeros((num_users, num_books))

# Populate the matrix with ratings
for row in ratings.itertuples():
    rating_matrix[row.user_id - 1, row.book_id - 1] = row.rating


Ratings Data Loaded
   user_id  book_id  rating
0        1      258       5
1        2     4081       4
2        2      260       5
3        2     9296       5
4        2     2318       3


In [ ]:
def matrix_factorization(R, K, steps=5000, alpha=0.0002, beta=0.02):
    num_users, num_books = R.shape
    P = np.random.rand(num_users, K)
    Q = np.random.rand(num_books, K)

    Q = Q.T  # Transpose Q for ease of calculation

    for step in range(steps):
        for i in range(num_users):
            for j in range(num_books):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])

                    for k in range(K):
                        P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] += alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        # Error calculation
        eR = np.dot(P, Q)
        e = np.sum((R[np.where(R > 0)] - eR[np.where(R > 0)]) ** 2) # We only take those not equal to 0 and square every such element in array. And finally get the sum of these elements
        # Regularization
        for i in range(num_users):
            for k in range(K):
                e += (beta / 2) * (P[i][k] ** 2)
        for j in range(num_books):
            for k in range(K):
                e += (beta / 2) * (Q[k][j] ** 2)

        if e < 0.001:
            break

    return P, Q.T

# Number of latent factors
K = 2

# Perform matrix factorization
P, Q = matrix_factorization(rating_matrix, K)

# Save matrix
np.savetxt(f'{o}/P.csv', P, delimiter=',')
np.savetxt(f'{o}/Q.csv', Q, delimiter=',')


In [ ]:
# Prediction and Error Calculation

# Predict the complete rating matrix
predicted_ratings = np.dot(P, Q.T)

# Calculate the prediction error
actual_ratings = rating_matrix[np.where(rating_matrix > 0)]
predicted_ratings = predicted_ratings[np.where(rating_matrix > 0)]
error = np.sqrt(np.mean((actual_ratings - predicted_ratings) ** 2))

print("Prediction Error:", error)